In [69]:
import pandas as pd
import numpy as np


In [99]:
df1 = pd.read_csv("../Data/eth_active_addresses_hr.csv")
df2 = pd.read_csv("../Data/eth_ohlcv_fees_tx_size_fail_rate_hr_new.csv")
df3 = pd.read_csv("../Data/eth_staking_inflow_hr.csv")
df4 = pd.read_csv("../Data/eth_exchange_deposite_withdraw_tx_hr.csv")
df5 = pd.read_csv("../Data/eth_exchange_netflow_usd_hr.csv")
df6 = pd.read_csv("../Data/eth_hourly_RV_new.csv")

In [100]:
for df in [df1, df2, df3, df4, df5, df6]:
    df['datetime'] = pd.to_datetime(df['datetime'])
df.head()

,datetime,realised_variance,realised_volatility
0,2024-12-28 00:00:00+00:00,0.000008,0.002832
1,2024-12-28 13:00:00+00:00,0.000013,0.003579
2,2024-12-29 01:00:00+00:00,0.000010,0.003137
3,2024-12-29 06:00:00+00:00,0.000010,0.003113
4,2024-12-29 14:00:00+00:00,0.000068,0.008248


In [101]:
# Add moving averages columns
df6.sort_values(by='datetime', inplace=True)
df6['RV_MA_1hr'] = df6['realised_volatility'].shift(1)
df6['RV_MA_3hr'] = df6['RV_MA_1hr'].rolling(window=3).mean()
df6['RV_MA_12hr'] = df6['RV_MA_1hr'].rolling(window=12).mean()
df6["vol_future"] = df6["realised_volatility"].shift(-1)
df6.drop(columns=["realised_volatility"], inplace=True)

# df between 2024-11-05 and 2025-10-12
start_date = "2024-11-05 00:00:00+00:00"
end_date = "2025-10-12 23:00:00+00:00"
df6 = df6[(df6['datetime'] >= start_date) & (df6['datetime'] <= end_date)]
df6.head()


,datetime,realised_variance,RV_MA_1hr,RV_MA_3hr,RV_MA_12hr,vol_future
3245,2024-11-05 00:00:00+00:00,0.000020,0.007329,0.010102,0.006048,0.009336
2207,2024-11-05 01:00:00+00:00,0.000087,0.004471,0.006700,0.006063,0.003825
4238,2024-11-05 02:00:00+00:00,0.000015,0.009336,0.007045,0.006538,0.003505
5257,2024-11-05 03:00:00+00:00,0.000012,0.003825,0.005877,0.006426,0.003399
1703,2024-11-05 04:00:00+00:00,0.000012,0.003505,0.005555,0.006192,0.003244


In [102]:
# Merge all on datetime
merged = (
    df6
    .merge(df1, on="datetime", how="left")
    .merge(df3, on="datetime", how="left")
    .merge(df4, on="datetime", how="left")
    .merge(df5, on="datetime", how="left")
    .merge(df2, on="datetime", how="left")
    .sort_values("datetime")
)

merged = merged[merged["datetime"] != "2025-10-13 00:00:00+00:00"]
merged['staking_inflow'] = merged['staking_inflow'].fillna(0)
merged.drop(columns=['realised_variance'], inplace=True)

print(merged.shape)
merged.head()

(8208, 21)


,datetime,RV_MA_1hr,RV_MA_3hr,RV_MA_12hr,vol_future,active_sending_addresses,active_receiving_addresses,staking_inflow,exchange_depositing_count,exchange_withdrawing_count,...,onchain_volume_usd,transaction_count,avg_gas_fee_usd,avg_priority_fee_usd,fail_rate_percent,open,low,high,close,hourly_return
0,2024-11-05 00:00:00+00:00,0.007329,0.010102,0.006048,0.009336,20914,12008,384.0,1883,6340,...,9.682857e+07,44719,1.817381,0.738825,2.4,2398.91,2398.91,2409.56,2405.13,0.003177
1,2024-11-05 01:00:00+00:00,0.004471,0.006700,0.006063,0.003825,20028,12945,63.0,1755,5877,...,9.693977e+07,44513,1.823433,0.819646,2.3,2402.45,2384.78,2410.94,2408.52,0.001408
2,2024-11-05 02:00:00+00:00,0.009336,0.007045,0.006538,0.003505,21410,12831,10529.0,2085,5846,...,1.304534e+08,45224,1.783517,0.724244,2.0,2411.90,2402.15,2413.11,2402.73,-0.002407
3,2024-11-05 03:00:00+00:00,0.003825,0.005877,0.006426,0.003399,20210,12622,286.0,1599,5429,...,9.107739e+07,42689,1.623861,0.716063,1.4,2405.17,2403.79,2417.69,2417.44,0.006104
4,2024-11-05 04:00:00+00:00,0.003505,0.005555,0.006192,0.003244,21076,10843,65.0,1575,5454,...,9.376725e+07,43063,1.346194,0.550501,1.4,2418.50,2418.50,2431.30,2428.68,0.004639


In [103]:
merged.tail()

,datetime,RV_MA_1hr,RV_MA_3hr,RV_MA_12hr,vol_future,active_sending_addresses,active_receiving_addresses,staking_inflow,exchange_depositing_count,exchange_withdrawing_count,...,onchain_volume_usd,transaction_count,avg_gas_fee_usd,avg_priority_fee_usd,fail_rate_percent,open,low,high,close,hourly_return
8203,2025-10-12 19:00:00+00:00,0.006496,0.012142,0.010524,0.007959,31227,19656,1.038051,2671,7753,...,2.849049e+08,69387,0.513560,0.368756,1.3,4111.71,4111.71,4144.37,4144.37,0.008199
8204,2025-10-12 20:00:00+00:00,0.006251,0.008313,0.010633,0.013743,29222,18801,0.000000,2374,8152,...,3.065971e+08,68653,0.837327,0.682312,1.3,4140.28,4135.07,4167.94,4167.94,0.005671
8205,2025-10-12 21:00:00+00:00,0.007959,0.006902,0.010962,0.007669,27708,16669,0.000000,2166,7152,...,3.288544e+08,64176,0.785752,0.634086,1.3,4145.22,4097.71,4158.51,4128.80,-0.009435
8206,2025-10-12 22:00:00+00:00,0.013743,0.009318,0.011700,0.009990,28854,17574,0.000000,2408,7749,...,2.991281e+08,64996,0.514463,0.412578,1.2,4135.83,4124.31,4150.47,4145.09,0.003938
8207,2025-10-12 23:00:00+00:00,0.007669,0.009790,0.011895,0.010211,22905,14923,0.000000,2182,5605,...,1.900689e+08,54465,0.488919,0.409657,1.1,4142.72,4118.27,4158.87,4153.71,0.002077


In [104]:
merged.columns.tolist()

['datetime',
 'RV_MA_1hr',
 'RV_MA_3hr',
 'RV_MA_12hr',
 'vol_future',
 'active_sending_addresses',
 'active_receiving_addresses',
 'staking_inflow',
 'exchange_depositing_count',
 'exchange_withdrawing_count',
 'exchange_netflow_usd',
 'onchain_volume_usd',
 'transaction_count',
 'avg_gas_fee_usd',
 'avg_priority_fee_usd',
 'fail_rate_percent',
 'open',
 'low',
 'high',
 'close',
 'hourly_return']

### Log transformation
for skew > 1:

if positively skewed: log

if there are negative values: signed log

if percentage (0-100): use power transform 

check skewness, if skew still > 1 after log:
power transform those features

In [105]:
from scipy.stats import skew
merged_log = merged.copy()
# Select numeric columns
numeric_cols = merged_log.select_dtypes(include=[np.number]).columns

# Compute skewness
skew_vals = merged_log[numeric_cols].apply(lambda x: skew(x.dropna()))
skew_df = pd.DataFrame({"feature": skew_vals.index, "skew": skew_vals.values})
skew_df = skew_df.sort_values(by="skew", ascending=False)

print(skew_df)


                       feature       skew
6               staking_inflow  22.338105
13        avg_priority_fee_usd  20.877469
12             avg_gas_fee_usd  12.340594
0                    RV_MA_1hr   7.116426
3                   vol_future   7.114231
1                    RV_MA_3hr   4.791243
7    exchange_depositing_count   4.440219
14           fail_rate_percent   3.887409
10          onchain_volume_usd   2.935166
2                   RV_MA_12hr   2.568031
9         exchange_netflow_usd   1.041357
4     active_sending_addresses   0.789280
8   exchange_withdrawing_count   0.770224
5   active_receiving_addresses   0.604319
11           transaction_count   0.477345
16                         low   0.118388
15                        open   0.115386
18                       close   0.114923
17                        high   0.112566
19               hourly_return  -0.233779


In [106]:
from sklearn.preprocessing import PowerTransformer
# log transform certain features
log_features = [
    "onchain_volume_usd",
    "avg_gas_fee_usd",
    "avg_priority_fee_usd",
    "staking_inflow",
    "exchange_depositing_count"
]

for col in log_features:
    merged_log[f'{col}_log'] = np.log1p(merged_log[col])
    merged_log = merged_log.drop(columns=[col])


# signed log
merged_log[f'{"exchange_netflow_usd"}_log'] = np.sign(merged_log["exchange_netflow_usd"]) * np.log1p(np.abs(merged_log["exchange_netflow_usd"]))
merged_log = merged_log.drop(columns=["exchange_netflow_usd"])


# Select numeric columns
numeric_cols = merged_log.select_dtypes(include=[np.number]).columns

# Compute skewness
skew_vals = merged_log[numeric_cols].apply(lambda x: skew(x.dropna()))
skew_df = pd.DataFrame({"feature": skew_vals.index, "skew": skew_vals.values})
skew_df = skew_df.sort_values(by="skew", ascending=False)

print(skew_df)

                          feature      skew
0                       RV_MA_1hr  7.116426
3                      vol_future  7.114231
1                       RV_MA_3hr  4.791243
8               fail_rate_percent  3.887409
2                      RV_MA_12hr  2.568031
16       avg_priority_fee_usd_log  1.995571
15            avg_gas_fee_usd_log  1.362627
4        active_sending_addresses  0.789280
18  exchange_depositing_count_log  0.772320
6      exchange_withdrawing_count  0.770224
5      active_receiving_addresses  0.604319
7               transaction_count  0.477345
10                            low  0.118388
9                            open  0.115386
12                          close  0.114923
11                           high  0.112566
14         onchain_volume_usd_log  0.057215
13                  hourly_return -0.233779
19       exchange_netflow_usd_log -0.341705
17             staking_inflow_log -0.787987


In [107]:
# power transform certain features
still_skew_col = [
    "fail_rate_percent",
    "avg_priority_fee_usd_log",
    "avg_gas_fee_usd_log"
]

pt = PowerTransformer(method='yeo-johnson')
merged_log[still_skew_col] = pt.fit_transform(merged_log[still_skew_col])

# Select numeric columns
numeric_cols = merged_log.select_dtypes(include=[np.number]).columns

# Compute skewness
skew_vals = merged_log[numeric_cols].apply(lambda x: skew(x.dropna()))
skew_df = pd.DataFrame({"feature": skew_vals.index, "skew": skew_vals.values})
skew_df = skew_df.sort_values(by="skew", ascending=False)

print(skew_df)

                          feature      skew
0                       RV_MA_1hr  7.116426
3                      vol_future  7.114231
1                       RV_MA_3hr  4.791243
2                      RV_MA_12hr  2.568031
4        active_sending_addresses  0.789280
18  exchange_depositing_count_log  0.772320
6      exchange_withdrawing_count  0.770224
5      active_receiving_addresses  0.604319
7               transaction_count  0.477345
15            avg_gas_fee_usd_log  0.258288
8               fail_rate_percent  0.159428
16       avg_priority_fee_usd_log  0.154917
10                            low  0.118388
9                            open  0.115386
12                          close  0.114923
11                           high  0.112566
14         onchain_volume_usd_log  0.057215
13                  hourly_return -0.233779
19       exchange_netflow_usd_log -0.341705
17             staking_inflow_log -0.787987


In [109]:
merged_log.isna().sum()

datetime                         0
RV_MA_1hr                        0
RV_MA_3hr                        0
RV_MA_12hr                       0
vol_future                       0
active_sending_addresses         0
active_receiving_addresses       0
exchange_withdrawing_count       0
transaction_count                0
fail_rate_percent                0
open                             0
low                              0
high                             0
close                            0
hourly_return                    0
onchain_volume_usd_log           0
avg_gas_fee_usd_log              0
avg_priority_fee_usd_log         0
staking_inflow_log               0
exchange_depositing_count_log    0
exchange_netflow_usd_log         0
dtype: int64

In [110]:
merged_log.to_csv("../Data/dune_eth_hour.csv", index=False)